# Merge in batch information and cropping/alignment parameters

In [1]:
library(data.table)

## Load data

In [2]:
df <- fread("1-OUT_paths_phases_timepoint_to_inspect_a4.csv")

In [3]:
batches <- fread("2-IN_CHECKLIST_phases_timepoint.csv")

In [4]:
parameters <- fread("2-IN_grid_crop_parameters.csv")

In [5]:
colnames(df)

[1] "dirs"            "phase"           "timepoint"       "phase_timepoint"
[5] "is_duplicated"

## Merge `df` of paths with batch info

In [6]:
colnames(batches)[1:2] <- c("phase", "timepoint")

In [7]:
df <- merge(df, batches)

In [8]:
head(df)

phase,timepoint,dirs,phase_timepoint,is_duplicated,group
GTAA,wk10,/media//gmobot/easystore/GWAS_Transformation/GTAA/wk10,GTAA_wk10,FALSE,Grids_post_shift/subset_b
GTAA,wk3,/media//gmobot/easystore/GWAS_Transformation/GTAA/wk3,GTAA_wk3,FALSE,Grids_post_shift/subset_a
GTAA,wk7,/media//gmobot/easystore/GWAS_Transformation/GTAA/wk7,GTAA_wk7,FALSE,Grids_post_shift/subset_a
GTAAA,wk10,/media//gmobot/Expansio_15/Expansion_15/GWAS_Transformation/GTAAA/wk10,GTAAA_wk10,FALSE,Grids_post_shift/subset_d
GTAAA,wk3,/media//gmobot/Expansio_15/Expansion_15/GWAS_Transformation/GTAAA/wk3,GTAAA_wk3,FALSE,Grids_post_shift/subset_d
GTAAA,wk7,/media//gmobot/Expansio_15/Expansion_15/GWAS_Transformation/GTAAA/wk7,GTAAA_wk7,FALSE,Grids_post_shift/subset_d


Do we have group info for everything?

In [9]:
df[is.na(df$group), ]

phase,timepoint,dirs,phase_timepoint,is_duplicated,group


In [10]:
df[which(df$group == ""), ]

phase,timepoint,dirs,phase_timepoint,is_duplicated,group


Yes.

## Find where each chroma grid in `parameters` file comes from

In [11]:
files <- list.files("/media/",
                    recursive = TRUE,
                    full.names = TRUE)

In [12]:
files <- files[grepl("GWAS_Transformation", files)]

In [13]:
files <- files[!grepl("desktop.ini", files)]
files <- files[!grepl("xlsx", files)]
files <- files[!grepl("roadband", files)]
files <- files[!grepl("rgb_test", files)]
files <- files[!grepl(".csv", files)]
files <- files[!grepl("duplicate", files)]
files <- files[!grepl("Not right", files)]
files <- files[!grepl("wrong", files)]
files <- files[!grepl("screenshots", files)]
files <- files[!grepl("subset", files)]
files <- files[!grepl("Grids", files)]
files <- files[!grepl("GRIDS", files)]
files <- files[!grepl(".txt", files)]
files <- files[!grepl("samples_for_annotation", files)]

Test for one

In [14]:
i <- 1

In [15]:
parameters$grid_file <- gsub("_pre-densenet", "", parameters$RGB[i])

In [16]:
files[grepl(parameters$grid_file[i], files)]

[1] "/media//gmobot/Elements_9/GWAS_Transformation/GTC/wk7/chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb.jpg"

Run for all

In [17]:
parameters$grid_file_rgb_path <- parameters$grid_file_hdr_path <- parameters$grid_file_prefix <- NA

In [18]:
for(i in 1:nrow(parameters)){
    print(i)
    parameters$grid_file_prefix[i] <- gsub("_pre-densenet", "", parameters$RGB[i])
    
    grid_file_rgb_path <- files[grepl(parameters$grid_file_prefix[i], files)][1]
    if(file.exists(grid_file_rgb_path)){
        parameters$grid_file_rgb_path[i] <- grid_file_rgb_path
    } else { 
        stop("Error! RGB File doesn't exist.")
    }
    
    grid_file_hdr_path <- gsub("_rgb.jpg", "_Fluorescence.hdr", parameters$grid_file_rgb_path[i])
    if(file.exists(grid_file_hdr_path)){
        parameters$grid_file_hdr_path[i] <- grid_file_hdr_path
    } else { 
        print(grid_file_hdr_path)
        stop("Error! HDR File doesn't exist.")

    }

}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6


In [19]:
head(parameters)

Shift_group,subset,Location,RGB,prealign_top,prealign_bottom,prealign_left,prealign_right,Align_file,D,GS,aligned_top,aligned_bottom,aligned_left,aligned_right,grid_file,grid_file_prefix,grid_file_hdr_path,grid_file_rgb_path
pre_shift,subset_a,GTC_7,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb_pre-densenet,439.0,1540,277,1725,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb_D0.52_GS0.62.jpg,0.52,0.62,99,1252,285,1227,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,/media//gmobot/Elements_9/GWAS_Transformation/GTC/wk7/chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_Fluorescence.hdr,/media//gmobot/Elements_9/GWAS_Transformation/GTC/wk7/chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb.jpg
pre_shift,subset_b,GTH_3,chromagrid_I5.0_F1.9_L100LED30_110504_0_0_0_rgb_pre-densenet,432.0,1527,270,1710,chromagrid_I5.0_F1.9_L100LED30_110504_0_0_0_rgb_D0.54_GS0.70.jpg,0.54,0.70,100,1252,283,1225,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_110504_0_0_0_rgb,/media//gmobot/Elements_8/GoogleDrive/Images_Elements_8/GWAS_Transformation/GTE/wk3/chromagrid_I5.0_F1.9_L100LED30_110504_0_0_0_Fluorescence.hdr,/media//gmobot/Elements_8/GoogleDrive/Images_Elements_8/GWAS_Transformation/GTE/wk3/chromagrid_I5.0_F1.9_L100LED30_110504_0_0_0_rgb.jpg
post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,469.5,1584,230,1688,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_D0.66_GS0.72.jpg,0.66,0.72,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg
post_shift,subset_b,GTDD_3,chromagrid_I5.0_F1.9_L100LED30_102651_0_0_0_rgb_pre-densenet,504.0,1613,267,1722,chromagrid_I5.0_F1.9_L100LED30_102651_0_0_0_rgb_D0.60_GS0.85.jpg,0.60,0.85,128,1284,346,1278,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_102651_0_0_0_rgb,/media//gmobot/Easystore_longerm_storage_2/GWAS_Transformation/GTDD/wk3/chromagrid_I5.0_F1.9_L100LED30_102651_0_0_0_Fluorescence.hdr,/media//gmobot/Easystore_longerm_storage_2/GWAS_Transformation/GTDD/wk3/chromagrid_I5.0_F1.9_L100LED30_102651_0_0_0_rgb.jpg
post_shift,subset_c,GTLL_3,chromagrid_F1.9_I5.0_L100LED30_102232_0_0_0_rgb_pre-densenet,501.0,1617,240,1701,chromagrid_F1.9_I5.0_L100LED30_102232_0_0_0_rgb_D0.55_GS0.65.jpg,0.55,0.65,109,1266,342,1280,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_F1.9_I5.0_L100LED30_102232_0_0_0_rgb,/media//gmobot/Elements_13/Elements_13/GWAS_Transformation/GTFF/wk10/chromagrid_F1.9_I5.0_L100LED30_102232_0_0_0_Fluorescence.hdr,/media//gmobot/Elements_13/Elements_13/GWAS_Transformation/GTFF/wk10/chromagrid_F1.9_I5.0_L100LED30_102232_0_0_0_rgb.jpg
post_shift,subset_d,GTUU_3,chromagrid_I5.0_F1.9_L100LED30_171422_0_0_0_rgb_pre-densenet,486.0,1602,221,1682,chromagrid_I5.0_F1.9_L100LED30_171422_0_0_0_rgb_D0.50_GS0.60.jpg,0.50,0.60,95,1246,330,1266,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_171422_0_0_0_rgb,/media//gmobot/Elements_14/Elements_14/GWAS_Transformation/GTUU/wk3/chromagrid_I5.0_F1.9_L100LED30_171422_0_0_0_Fluorescence.hdr,/media//gmobot/Elements_14/Elements_14/GWAS_Transformation/GTUU/wk3/chromagrid_I5.0_F1.9_L100LED30_171422_0_0_0_rgb.jpg


In [21]:
library(stringr)

In [22]:
phase_timepoint <- str_split_fixed(dirname(parameters$grid_file_rgb_path),
                                   "GWAS_Transformation",
                                   2)[, 2]

In [23]:
phase_timepoint

[1] "/GTC/wk7"   "/GTE/wk3"   "/GTW/wk10"  "/GTDD/wk3"  "/GTFF/wk10"
[6] "/GTUU/wk3"

In [24]:
parameters$phase_timepoint_of_grid_standard <- gsub("/G", "G", phase_timepoint)

## Merge `df` of paths, batch info with parameters

In [26]:
df$group <- gsub("Grids_", "", df$group)

In [27]:
parameters$group <- paste0(parameters$Shift_group, "/", parameters$subset)

In [28]:
head(parameters$group)

[1] "pre_shift/subset_a"  "pre_shift/subset_b"  "post_shift/subset_a"
[4] "post_shift/subset_b" "post_shift/subset_c" "post_shift/subset_d"

In [29]:
df <- merge(df, parameters, by = "group")

In [30]:
head(df)

group,phase,timepoint,dirs,phase_timepoint,is_duplicated,Shift_group,subset,Location,RGB,...,GS,aligned_top,aligned_bottom,aligned_left,aligned_right,grid_file,grid_file_prefix,grid_file_hdr_path,grid_file_rgb_path,phase_timepoint_of_grid_standard
post_shift/subset_a,GTAA,wk3,/media//gmobot/easystore/GWAS_Transformation/GTAA/wk3,GTAA_wk3,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,0.72,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10
post_shift/subset_a,GTAA,wk7,/media//gmobot/easystore/GWAS_Transformation/GTAA/wk7,GTAA_wk7,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,0.72,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10
post_shift/subset_a,GTO,wk10,/media//gmobot/Seagate Portable Drive/GWAS_Transformation/GTO/wk10,GTO_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,0.72,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10
post_shift/subset_a,GTP,wk10,/media//gmobot/New Volume/GWAS_Transformation/GTP/wk10,GTP_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,0.72,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10
post_shift/subset_a,GTQ,wk10,/media//gmobot/MyBook_16/MyBook_16/GWAS_Transformation/GTQ/wk10,GTQ_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,0.72,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10
post_shift/subset_a,GTQ,wk10,/media//gmobot/MyBook_16/MyBook_16/GWAS_Transformation/GTQ/wk10,GTQ_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,0.72,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10


## Add randomization datasheets

In [38]:
rd <- list.files("Randomization_datasheets/",
                 pattern = "labels",
                 full.names = TRUE)

In [39]:
df$randomization_datasheet <- NA

In [40]:
for(i in 1:nrow(df)){
    phase_ <- paste0(df$phase[i], "_")
    this_rd <- rd[which(grepl(phase_, rd))]
    if(length(this_rd) > 1){
        stop("Alert! More than one datashet for this phase?!")
    }
    if(length(this_rd) == 1){
        df$randomization_datasheet[i] <- this_rd
    }
}

In [41]:
head(df)

group,phase,timepoint,dirs,phase_timepoint,is_duplicated,Shift_group,subset,Location,RGB,...,aligned_top,aligned_bottom,aligned_left,aligned_right,grid_file,grid_file_prefix,grid_file_hdr_path,grid_file_rgb_path,phase_timepoint_of_grid_standard,randomization_datasheet
post_shift/subset_a,GTAA,wk3,/media//gmobot/easystore/GWAS_Transformation/GTAA/wk3,GTAA_wk3,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10,Randomization_datasheets//GTAA_16_genotypes_6.30.21-7.02.21_labels.xlsx
post_shift/subset_a,GTAA,wk7,/media//gmobot/easystore/GWAS_Transformation/GTAA/wk7,GTAA_wk7,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10,Randomization_datasheets//GTAA_16_genotypes_6.30.21-7.02.21_labels.xlsx
post_shift/subset_a,GTO,wk10,/media//gmobot/Seagate Portable Drive/GWAS_Transformation/GTO/wk10,GTO_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10,Randomization_datasheets//GTO_16_genotypes_03.01.21-03.03.21_labels.xlsx
post_shift/subset_a,GTP,wk10,/media//gmobot/New Volume/GWAS_Transformation/GTP/wk10,GTP_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10,Randomization_datasheets//GTP_32_genotypes_03.09.21-03.12.21_labels.xlsx
post_shift/subset_a,GTQ,wk10,/media//gmobot/MyBook_16/MyBook_16/GWAS_Transformation/GTQ/wk10,GTQ_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10,Randomization_datasheets//GTQ_32_genotypes_03.15.21-03.18.21_labels.xlsx
post_shift/subset_a,GTQ,wk10,/media//gmobot/MyBook_16/MyBook_16/GWAS_Transformation/GTQ/wk10,GTQ_wk10,FALSE,post_shift,subset_a,GTW_10,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb_pre-densenet,...,98,1254,280,1228,chromagrid_I5.0_F1.9_L100LED30_133853_0_0_0_rgb,chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_Fluorescence.hdr,/media//gmobot/easystore/GWAS_Transformation/GTW/wk10/chromagrid_I5.0_F1.9_L100LED30_124643_0_0_0_rgb.jpg,GTW/wk10,Randomization_datasheets//GTQ_32_genotypes_03.15.21-03.18.21_labels.xlsx
